## Import other Libraries

In [1]:
import pandas as pd
import numpy as np

## Read in data

In [2]:
top_25 = pd.read_csv('./processed_data_1/cleaned_building_permits&pop_top_25.csv')
york = pd.read_csv('./processed_data_1/cleaned_building_permits&pop_york_region.csv')

In [3]:
top_25.head();

In [4]:
york.head();

## Check General Information and Data Types

In [5]:
top_25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   csd                                   500 non-null    object 
 1   province_or_territory                 500 non-null    object 
 2   year                                  500 non-null    int64  
 3   value_nonresidential_commercial       500 non-null    float64
 4   value_nonresidential_industrial       500 non-null    float64
 5   value_nonresidential_institutional    500 non-null    float64
 6   value_residential_single_single       500 non-null    float64
 7   value_residential_single_mobile       500 non-null    float64
 8   value_residential_single_cottage      500 non-null    float64
 9   value_residential_multiple_double     500 non-null    float64
 10  value_residential_multiple_row        500 non-null    float64
 11  value_residential_m

In [6]:
york.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   csd                                   140 non-null    object 
 1   province_or_territory                 140 non-null    object 
 2   year                                  140 non-null    int64  
 3   value_nonresidential_commercial       140 non-null    float64
 4   value_nonresidential_industrial       140 non-null    float64
 5   value_nonresidential_institutional    140 non-null    float64
 6   value_residential_single_single       140 non-null    float64
 7   value_residential_single_mobile       140 non-null    float64
 8   value_residential_single_cottage      140 non-null    float64
 9   value_residential_multiple_double     140 non-null    float64
 10  value_residential_multiple_row        140 non-null    float64
 11  value_residential_m

In [7]:
top_25.shape

(500, 19)

In [8]:
york.shape

(140, 19)

In [9]:
top_25.describe().T;

In [10]:
york.describe().T;

## Feature Engineering

### Generate totol unit and value

In [11]:
def generate_sum(df):
    val_cols = [i for i in list(df.columns) if i.startswith('value')]
    unit_cols = [i for i in list(df.columns) if i.startswith('units')]
    val_sum_res_cols = [i for i in list(df.columns) if i.startswith('value_residential')]
    val_sum_nonres_cols = [i for i in list(df.columns) if i.startswith('value_nonresidential')]
    
    df['val_sum'] = df[val_cols].sum(axis=1)
    
    df['val_sum_res'] = df[val_sum_res_cols].sum(axis=1)
    df['val_sum_nonres'] = df[val_sum_nonres_cols].sum(axis=1)
    
    # For units, the data source only provides number of units for residential
    df['unit_sum_res'] = df[unit_cols].sum(axis=1)
    return df

In [12]:
generate_sum(top_25)
generate_sum(york);

### Generate unit and value per capita

In [13]:
def generate_pc(df):
    cols = [i for i in list(df.columns) if i not in ['csd', 'province_or_territory', 'year', 'population']]
    for col in cols:
        df[f'pc_{col}'] = df[col] / df['population']
    return df

In [14]:
generate_pc(top_25)
generate_pc(york);

### Generate percentage of each unit and value types

In [15]:
def generate_percent_unit(df):
    cols = [i for i in list(df.columns) if i.startswith('units')]
    for col in cols:
        df[f'percent_{col}'] = df[col] / df['unit_sum_res']
    return df

In [16]:
generate_percent_unit(top_25)
generate_percent_unit(york);

In [17]:
# # Check if percentages add up to 100%
# percent_units_cols = [i for i in list(top_25.columns) if i.startswith('percent_units')]
# top_25['sum'] = top_25[percent_units_cols].sum(axis=1)
# pd.set_option('display.max_rows', 500)
# top_25[top_25['sum'] != 1.0]

In [18]:
def generate_percent_value(df):
    cols = [i for i in list(df.columns) if i.startswith('value')]
    for col in cols:
        df[f'percent_{col}'] = df[col] / df['val_sum']
    return df

In [19]:
generate_percent_value(top_25)
generate_percent_value(york);

In [20]:
# # Check if percentages add up to 100%
# percent_val_cols = [i for i in list(top_25.columns) if i.startswith('percent_value')]
# top_25['sum'] = top_25[percent_val_cols].sum(axis=1)
# pd.set_option('display.max_rows', 500)
# top_25[top_25['sum'] != 1.0]

### Generate rolling sum and avg

#### Top 25

In [21]:
# Reference: https://stackoverflow.com/questions/53339021/python-pandas-calculate-moving-average-within-group
top_25['rolling_avg_val'] = top_25.groupby('csd').rolling(3)['val_sum'].mean().reset_index(drop=True)

In [22]:
top_25['rolling_sum_val'] = top_25.groupby('csd').rolling(3)['val_sum'].sum().reset_index(drop=True)

In [23]:
top_25['rolling_avg_unit'] = top_25.groupby('csd').rolling(3)['unit_sum_res'].mean().reset_index(drop=True)

In [24]:
top_25['rolling_sum_unit'] = top_25.groupby('csd').rolling(3)['unit_sum_res'].sum().reset_index(drop=True)

In [25]:
# pd.set_option('display.max_rows', 500)
# top_25.head(25).T

#### York region

In [26]:
# Reference: https://stackoverflow.com/questions/53339021/python-pandas-calculate-moving-average-within-group
york['rolling_avg_val'] = york.groupby('csd').rolling(3)['val_sum'].mean().reset_index(drop=True)

In [27]:
york['rolling_sum_val'] = york.groupby('csd').rolling(3)['val_sum'].sum().reset_index(drop=True)

In [28]:
york['rolling_avg_unit'] = york.groupby('csd').rolling(3)['unit_sum_res'].mean().reset_index(drop=True)

In [29]:
york['rolling_sum_unit'] = york.groupby('csd').rolling(3)['unit_sum_res'].sum().reset_index(drop=True)

In [30]:
# york.head(25)

## Export as csv

In [31]:
# top_25.to_csv("eda_top_25.csv", index=False)
# york.to_csv("eda_york.csv", index=False)